# Theoretischer Hintergrund

Wir betrachten eine Teilmenge $Q(d)$ des $d$-dimensionalen Euklidischen Raums $E(d)$, bestehend aus den Punkten, deren Kartesische Koordinaten rationale Zahlen sind. 

Ein Punkt in $Q(d)$ kann als Liste von $d+1$ ganzen Zahlen $[x_0, x_1, . . . , x_d ]$ mit $x_d \neq 0$, dargestellt werden (homogene Koordinaten). 
Diese Liste repräsentiert die $d$ rationalen Kartesischen Koordinaten $[x_0/x_d , x_1/x_d , . . . , x_{d−1}/xd]$
.

Ein Punkt in $Q(d)$ ist eine Liste von ganzen Zahlen:
```Haskell
type Point = [Integer]
```

## Simplex
Ein $d$-Simplex ist die Verallgemeinerung eines Dreiecks in $d$-Dimensionen.
![Simplex](img/simplex.PNG)
Ein $d$-Simplex kann als Liste von $d+1$ Punkten in $Q(d)$ und einem Wert $+1$ oder $-1$, der die Orientierung des Simplex angibt, repräsentiert werden:
```Haskell

type Simplex = ([Point], Int)
```

## Konvexe Menge
Jedes $d$-Simplex $smp$ bestimmt eine konvexe Menge $CS(smp)$ in $Q(d)$. 

Eine Teilmenge des euklidischen Raums (geometrische Figur) ist konvex, wenn für je zwei beliebige Punkte, die zur Menge gehören, auch stets deren Verbindungsstrecke ganz in der Menge liegt
![konvexe Menge](img/cm1.PNG)

Der Bereich enthält die Punkte innerhalb von $smp$ und auf dessen Grenze.

Um zu bestimmen ob ein Punkt innerhalb oder auf $CS(smp)$ liegt, müssen die Facetten von $smp$ berechnet werden.

## Facette
Die Facette eines $d$-Simplex kann als Liste von $d$ Punkten und einer dazugehörigen Orientierung (abgeleitet von dem Simplex), dargestellt werden. 
Beispielsweise sind die Facetten einer Kante ihre zwei Endpunkte, die Facetten eines Dreiecks sind ihre drei Kanten.

```Haskell

type Facet = ([Point], Int).
```

### Beispiel
Ein Dreieck $[a,b,c]$ mit positiver Orientierung im $Q(2)$ hat drei Facetten $([b, c], +1)$, $([a, c],−1)$, $([a, b], +1)$. 
Ein Punkt $p$ befindet sich innerhalb von $[a,b,c]$, wenn die drei Simplexe $ [p, b, c]$, $[p, a, c]$ und $[p, a, b]$ die Orientierungen $+1$, $−1$ und $+1$ haben. 

Implikation:
- $p$ befindet sich links von der Kante $b$ nach $c$
- $p$ befindet sich rechts von der Kante $a$ nach $c$
- $p$ befindet sich links von der Kante $a$ nach $b$

![Triangle](img/triangle1.PNG)

## Konvexe Hülle
Die konvexe Hülle $CH(vs)$ einer Punktmenge $vs$ in $Q(d)$ entspricht einem Bereich in $Q(d)$ und kann wie folgt definiert werden:
- $CH(vs)$ ist die Vereinigung der Mengen $CS(smp)$ für alle $d$-Simplexe $smp$ bestimmt durch Punkte von $vs$

In $Q(2)$ bestimmt sich die konvexe Hülle von $vs$ durch die Vereinigungsmenge der Bereiche, die durch alle Dreiecke deren Eckpunkte innerhalb von $vs$ liegen. 

Die Menge $CH(vs)$ lässt sich dementsprechend durch folgendes Prädikat charakterisieren:
```Haskell

insideCH :: [Point] -> Point -> Bool
insideCH vs p = or [insideCS smp p | smp <- simplexes vs]
```
**Um die CH zu berechnen müssen wir schauen ob der Punkt in einer konvexen Menge liegt. Dafür müssen wir Simplexe aus unserer Punktmenge bilden**

![Konvexe Hülle](img/ch.PNG)
Das Bild zeigt die konvexe Hülle von Punkten in der Ebene. Sie besteht aus dem schwarz umrandeten Gebiet (inklusive Rand).

## Hürden
- `tuples` Funktion muss selbst implementiert werden
- Bug in `op :: Facet -> [Facet] -> [Facet]`:
    - Das Problem: `op smp [] = []`
    - Lösung: `op smp [] = [smp]`
- QuickCheck infinite Loop
    - Das Problem: `points d (n + 1) = do {p <- point d; ps <- points d n; return (p : ps)}`
    - Lösung: `points d n  = do { p <- point d; ps <- points d (n-1); return (p:ps) }` (statt vorne n + 1 nur das n übergeben und hinten 1 abziehen)
- Bug in `partition` Funktion:
     - Das Problem: `Just [smp] -> foldl update [smp] (vs \\ vertices smp)`
     - Lösung: `Just smp -> foldl update [smp] (vs \\ vertices smp)`

# Determinante
Die Determinantenberechnungen stammen aus aus der "Functional Pearl: Three ways of computing determinants".

In [ ]:
-- Die Unterdeterminanten
minors :: [a] -> [[a]]
minors [ ] = [ ]
minors (x : xs) = xs : map (x :) (minors xs)

In [ ]:
minors "abcd"

In [ ]:
det :: [[Integer]] -> Integer
det [[x ]] = x
det xss = foldr1 (-) (zipWith (*) col1 (map det (minors cols)))
          where col1 = map head xss
                cols = map tail xss

In [ ]:
matrix = [[2,2,1],[9,2,5],[3,7,3]]
det matrix

In [ ]:
-- A more fancy Determinant calculation based on Integer Division
det :: [[Integer]] -> Integer
det [[x]] = x
det xss =
  case break ((/= 0) . head) xss of
   (yss, [ ]) -> 0
   (yss, zs : zss) -> let x = det(condense(zs : yss ++ zss))
                          d = head zs ^ (length xss - 2)
                          y = x `div` d
                      in if even (length yss) then y else -y

condense = map (map det . pair . uncurry zip) . pair
           where pair (x : xs) = map ((, ) x ) xs
                 det ((a, b), (c, d)) = a * d - b * c

In [ ]:
matrix = [[2,2,1],[9,2,5],[3,7,3]]
det matrix

# Konvexe Hülle

In [ ]:
import Data.List  

In [ ]:
-- Type synonym (basically just an alias)
type Point = [Integer]
type Facet = ([Point], Int)
type Simplex = ([Point], Int)

In [ ]:
dimension :: Point -> Int
dimension ps = length ps - 1

orientation :: [Point] -> Int
orientation = fromIntegral . signum . det

In [ ]:
facets :: Simplex -> [Facet]
facets (us, b) = zip (minors us) (cycle [b, -b])

minors :: [a] -> [[a]]
minors [] = []
minors (x : xs) = xs : map (x :) (minors xs)

In [ ]:
insideCS :: Simplex -> Point -> Bool
-- The generator (us, b) <- facets smp feeds each Facet in turn to the left-hand expression
insideCS smp p = and [0 <= b * orientation (p : us) | (us, b) <- facets smp]

In [ ]:
-- The value of tuples n vs is a list of all n-tuples of vs; that is, all subsequences
-- of vs of length n. The definition of tuples is left as an exercise.
tuples' :: Int -> [a] -> [[a]]
tuples' n vs = filter (\x -> length x == n) (subsequences vs)

In [ ]:
tuples :: Int -> [a] -> [[a]]
tuples n = filter ((==n) . length) . subsequences

tuples 2 [[1,2], [3,4], [5,6], [7,8]]

In [ ]:
insideCH :: [Point] -> Point -> Bool
insideCH vs p = or [insideCS smp p | smp <- simplexes vs]

simplexes :: [Point] -> [Simplex]
simplexes vs = [(us, b) | us <- tuples (d + 1) vs,
                          let b = orientation us, b /= 0]
               where d = dimension (head vs)

In [ ]:
insideCH :: [Point] -> Point -> Bool
insideCH vs p = if null smps
                then False
                else or [insideCS smp p | smp <- smps]
                where smps = simplexes vs

simplexes :: [Point] -> [Simplex]
simplexes vs = [(us, b) | us <- tuples (d + 1) vs,
                          let b = orientation us, b /= 0]
               where d = dimension (head vs)

In [ ]:
-- The vertices of a simplex are listed by:
vertices :: Simplex -> [Point]
vertices = sort . fst

-- fst = erstes Element in einem Tuple
-- sort sortiert die Liste der Punkte im Simplex

-- z.B: 
-- Input:     vertices ([[-10,4], [10,2], [3,3], [1,1]], 1)
-- Output: [[-10,4],[1,1],[3,3],[10,2]]

-- --> Das ist im Endeffekt nur eine Methode aus dem 
-- Tuple eine sortierte Liste der Punkte zu extrahieren und die Orientierung wegzuwerfen

## Find a Simplex

In [ ]:
-- The inefficient variant
-- findSimplex vs = if null smps then Nothing else Just (head smps)
--                  where smps = simplexes vs

In [ ]:
submatrices k vs = map (++[last vs]) (tuples k (init vs))
degenerate k = all (==0) . map det . submatrices k . transpose

In [ ]:
-- The more efficient variant
-- TODO: Why is this more efficient???
findSimplex :: [Point] -> Maybe Simplex
findSimplex [] = Nothing
findSimplex (v : vs) = search (length v - 1) 1 [v] vs

search d k us vs
  | k == d + 1 = Just (us, orientation us)
  | null vs = Nothing
  | degenerate k (v : us) = search d k us (tail vs)
  | otherwise = search d (k + 1) (v : us) (tail vs)
    where v = head vs


## Update

In [ ]:
external :: [Simplex] -> [Facet]
external = foldr op [] . sort . concatMap facets

-- Originalversion mit Bug
-- op smp [] = []
-- op smp (smp' : smps) = if vertices smp == vertices smp' then smps
--                        else smp : smp' : smps
                       
op :: Facet -> [Facet] -> [Facet]
op smp []     = [smp]  -- bug fixed
op smp (smp' : smps) = if vertices smp == vertices smp' then smps 
                       else smp : smp' : smps

In [ ]:
visible :: Point -> [Facet] -> [Facet]
visible v fs = [(us, b) | (us, b) <- fs, b * orientation (v : us) < 0]

In [ ]:
newSimplex :: Point -> Facet -> Simplex
newSimplex v (us, b) = (v : us, -b)

In [ ]:
update :: [Simplex] -> Point -> [Simplex]
update smps v = smps ++ map (newSimplex v) (visible v (external smps))

## Incremental Algorithm

In [ ]:
insideCH' :: [Point] -> Point -> Bool
insideCH' vs p = or [insideCS smp p | smp <- partition vs] 

partition :: [Point] -> [Simplex]
partition vs
    = case findSimplex vs of
        Nothing -> []
        Just smp -> foldl update [smp] (vs \\ vertices smp)
        -- Laut Paper: Just [smp] -> foldl update [smp] (vs \\ vertices smp)
        
-- Der \\ Operator bildet die Listendifferenz z.B.:
-- Input: [1,2,3,4] \\ [2,3]
-- Output: [1,4]

-- foldl nimm das zweite Argument und ein Element der Liste und wendet die jeweilige Funktion an hier: (/)
-- Input: foldl (/) 64 [4,2,4]
-- Output: 2.0

## Test

In [ ]:
pts = [[2, 4, 1],
       [4, 7, 1],
       [2, 2, 1],
       [9, 6, 1],
       [1, 9, 1],
       [5, 7, 1],
       [0, 9, 1],
       [5, 1, 1],
       [0, 2, 1],
       [8, 4, 1],
       [6, 6, 1],
       [3, 6, 1],
       [1, 9, 1],
       [0, 5, 1],
       [0, 3, 1],
       [9, 4, 1],
       [9, 5, 1],
       [6, 0, 1],
       [6, 7, 1],
       [3, 6, 1],
       [6, 8, 1],
       [1, 0, 1],
       [7, 8, 1],
       [9, 6, 1],
       [6, 9, 1],
       [2, 9, 1],
       [8, 8, 1],
       [6, 7, 1],
       [9, 8, 1],
       [0, 4, 1]]

In [ ]:
testPts = [[5, 5, 1], [0, 1, 1], [1, 4, 1], [50, 50, 1]]

r = map(\x -> insideCH' pts x) testPts
r

In [ ]:
r = map(\x -> insideCH pts x) testPts
r

In [ ]:
length r

In [ ]:
filter (\x -> x == True) r

In [ ]:
filtermask f [] [] = []
filtermask f (fm:rm) (fd:rd)
  |f fm fd = fd:filtermask f rm rd
  |otherwise = filtermask f rm rd

In [ ]:
filtermask (\m d -> m == True) r pts

In [ ]:
testPts = [[10, 10], [1,1], [0,4], [50,50], [3,6], [-1, 5]]
r = map(\x -> insideCH' pts x) testPts
r

In [ ]:
-- Dont do this --> infinite loop
-- insideCH pts [-20, -20]

In [ ]:
-- Collinear Points
collinearPts = [[0,0,1],[0,0,1],[0,0,1],[-1,-1,1]]
insideCH collinearPts [1,0,1]
insideCH' collinearPts [1,0,1]

## An Improvement

In [ ]:
-- insideCH'' vs p = if null fs
--     then False
--    else and [0 <= b*orientation (p:us) | (us, b) <- fs]
--    where fs = faces vs
 
 
-- Bug fix: Here we have to call the faces function on vs
insideCH'' vs p = not (null fs) && and [0 <= b * orientation (p : us) | (us, b) <- fs]
                  where fs = faces vs
 
-- Bug fix: Here we have to replace Just [smp] -> foldl update' (facets smp) (vs \\ vertices vs) by the below expression
faces :: [Point] -> [Facet]
faces vs = case findSimplex vs of
           Nothing -> []
           Just smp  -> foldl update' (facets smp) (vs \\ vertices smp)
 
update' :: [Facet] -> Point -> [Facet]
update' fs v = (fs \\ fs') ++ map (newFacet v) (external fs')
               where fs' = visible v fs
 
newFacet v (us, b) = (v:us, b)

In [ ]:
-- external (update smps v) = update' (external smps) v

## QuickCheck

In [ ]:
import Test.QuickCheck

In [ ]:
-- QuickCheck - working version
point :: Int -> Gen [Integer]
point d = do { xs <- vector d; return (xs ++ [1]) }
 
points :: Int -> Int -> Gen [[Integer]]
points d 0  = return []
points d n  = do { p <- point d; ps <- points d (n-1); return (p:ps) }
 
prop_Hull :: Int -> Int -> Property
prop_Hull d n =
    forAll (points d n) $ \vs ->
    forAll (point d) $ \v ->
    insideCH vs v == insideCH' vs v && insideCH vs v == insideCH'' vs v
 
qc = quickCheck (prop_Hull 2 4)
qc

In [ ]:
verboseCheck (prop_Hull 2 4)